In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.mixture import GaussianMixture
from itertools import product
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)
from sklearn.covariance import EllipticEnvelope

c:\Users\asgar\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read data
data = pd.read_csv("data.csv")
data = data.drop(
    [
        "timestamp",
    ],
    axis=1,
)
data.reset_index(drop=True, inplace=True)
data["is_anomaly"] = 0

In [ ]:
# Define helper functions
def normalize_data(data):
    scaling_rules = {
        "cpu_temperature": (25.0, 100.0),
        "gpu_temperature": (25.0, 100.0),
        "cpu_speed": (0.60, 4.80),
        "cpu_fan_speed": (0.0, 7500.0),
        "gpu_fan_speed": (0.0, 7500.0),
        "disk_reads": (0.0, 3000.0),
        "disk_writes": (0.0, 3000.0),
        "network_bytes_sent": (0.0, 200000.0),
        "network_bytes_received": (0.0, 5000000.0),
        "network_packets_sent": (0.0, 500.0),
        "network_packets_received": (0.0, 4000.0),
    }

    for column, (min_val, max_val) in scaling_rules.items():
        if column in data.columns:
            if data[column].min() >= 0 and data[column].max() <= 1:
                continue

            data[column] = (data[column] - min_val) / (max_val - min_val)
            data[column] = data[column].clip(0, 1)

    return data


def synthesize_anomaly(data_row, scenario=None):
    data_row = data_row.copy()

    data_row["is_anomaly"] = 1

    if scenario == None:
        scenario = np.random.choice(
            [
                "thermal_overload",
                "cpu_overload",
                "disk_filesystem_abuse",
                "network_abuse",
            ]
        )

    if scenario == "thermal_overload":
        data_row["cpu_temperature"] += 15
        data_row["cpu_temperature"] = min(data_row["cpu_temperature"], 100)

        data_row["gpu_temperature"] += 15
        data_row["gpu_temperature"] = min(data_row["gpu_temperature"], 100)

        data_row["cpu_fan_speed"] += 1000
        data_row["gpu_fan_speed"] += 1000

        data_row["cpu_fan_speed"] = min(data_row["cpu_fan_speed"], 7500)
        data_row["gpu_fan_speed"] = min(data_row["gpu_fan_speed"], 7500)

    elif scenario == "disk_filesystem_abuse":
        data_row["cpu_usage"] *= 1.20
        data_row["cpu_speed"] *= 1.30

        data_row["disk_reads"] += 500
        data_row["disk_writes"] += 500

        data_row["disk_reads"] = min(data_row["disk_reads"], 3000)
        data_row["disk_writes"] = min(data_row["disk_writes"], 3000)

    elif scenario == "network_abuse":
        data_row["cpu_usage"] *= 1.20
        data_row["cpu_speed"] *= 1.30

        data_row["network_bytes_sent"] += 10000
        data_row["network_bytes_received"] += 250000

        data_row["network_packets_sent"] += 120
        data_row["network_packets_received"] += 600

        data_row["cpu_usage"] = min(data_row["cpu_usage"], 1.0)
        data_row["cpu_speed"] = min(data_row["cpu_speed"], 4.80)

        data_row["network_bytes_sent"] = min(data_row["network_bytes_sent"], 200000)
        data_row["network_bytes_received"] = min(
            data_row["network_bytes_received"], 5000000
        )

        data_row["network_packets_sent"] = min(data_row["network_packets_sent"], 500)
        data_row["network_packets_received"] = min(
            data_row["network_packets_received"], 4000
        )

    elif scenario == "cpu_overload":
        data_row["cpu_usage"] *= 1.20
        data_row["cpu_speed"] *= 1.30
        data_row["cpu_fan_speed"] += 1000

        data_row["cpu_usage"] = min(data_row["cpu_usage"], 1.0)
        data_row["cpu_speed"] = min(data_row["cpu_speed"], 4.80)
        data_row["cpu_fan_speed"] = min(data_row["cpu_fan_speed"], 7500)

    return data_row

In [5]:
learning_period_seconds = 60 * 60 * 1  # 1 hour
retraining_period_seconds = 60 * 60 * 0.5  # 30 minutes

# Each row is collected every 5 seconds
LEARNING_PERIOD_ROWS_COUNT = learning_period_seconds // 5
RETRAINING_PERIOD_ROWS_COUNT = retraining_period_seconds // 5
TOTAL_ROWS_COUNT = len(data)
random_state = 42

np.random.seed(random_state)


# Generate Synthetic Anomalies
i = LEARNING_PERIOD_ROWS_COUNT
while i < TOTAL_ROWS_COUNT:
    rand_number = np.random.randint(0, 2)

    if rand_number == 1:
        anomaly_count = np.random.randint(3, 7)
        for j in range(i, i + anomaly_count):
            if j < TOTAL_ROWS_COUNT:
                data.loc[j] = synthesize_anomaly(data.iloc[j])
        i += anomaly_count

    i += 15

TOTAL_ANOMALY_ROWS_COUNT = data["is_anomaly"].sum()

print(f"Total rows: {TOTAL_ROWS_COUNT}")
print(f"Learning period rows: {LEARNING_PERIOD_ROWS_COUNT}")
print(f"Retraining period rows: {RETRAINING_PERIOD_ROWS_COUNT}")
print(f"Total anomaly rows: {TOTAL_ANOMALY_ROWS_COUNT}")

# Normalize data
data = normalize_data(data)
data_labels = data["is_anomaly"]
data = data.drop(["is_anomaly"], axis=1)

Total rows: 1492
Learning period rows: 720
Retraining period rows: 360.0
Total anomaly rows: 80


In [ ]:

class OneClassSVMWrapper:
    def __init__(self, kernel="rbf", nu=0.05, gamma="scale", features=[]):
        self.model = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)
        self.features = features

    def fit(self, X):
        self.X = X[self.features]
        self.model.fit(self.X)

    def predict(self, X):
        self.X = X[self.features]
        # One-Class SVM returns +1 for inliers, -1 for outliers.
        # We map outliers (-1) to anomaly (1) and inliers (+1) to normal (0).
        preds = self.model.predict(self.X)
        return [1 if p == -1 else 0 for p in preds]

    def get_name(self):
        return "OneClassSVM"


class LOFWrapper:
    def __init__(self, n_neighbors=20, novelty=True, contamination=0.05, features=[]):
        # novelty=True allows LOF to be used for out-of-sample predictions.
        self.model = LocalOutlierFactor(
            n_neighbors=n_neighbors, novelty=novelty, contamination=contamination
        )
        self.features = features

    def fit(self, X):
        self.X = X[self.features]
        self.model.fit(self.X)

    def predict(self, X):
        self.X = X[self.features]
        # LOF returns 1 for inliers, -1 for outliers.
        preds = self.model.predict(self.X)
        return [1 if p == -1 else 0 for p in preds]

    def get_name(self):
        return "LocalOutlierFactor"


class EllipticEnvelopeWrapper:
    def __init__(self, contamination=0.1, support_fraction=0.8, features=[]):
        self.model = EllipticEnvelope(
            contamination=contamination,
            support_fraction=support_fraction,
            random_state=42,
        )
        self.features = features

    def fit(self, X):
        self.X = X[self.features]
        self.model.fit(self.X)

    def predict(self, X):
        self.X = X[self.features]
        # EllipticEnvelope returns +1 for inliers and -1 for outliers.
        preds = self.model.predict(self.X)
        return [1 if p == -1 else 0 for p in preds]

    def get_name(self):
        return "EllipticEnvelopeWrapper"


class GaussianMixtureWrapper:
    def __init__(
        self, n_components=1, covariance_type="full", threshold=None, features=[]
    ):
        self.features = features
        self.model = GaussianMixture(
            n_components=n_components, covariance_type=covariance_type
        )
        self.threshold = threshold 
        self.fitted_scores = None

    def fit(self, X):
        self.X = X[self.features]
        self.model.fit(self.X)
        # Compute log-likelihood for training data.
        scores = self.model.score_samples(self.X)
        self.fitted_scores = scores
        if self.threshold is None:
            # For example, set threshold as (mean - std) of the training scores.
            self.threshold = scores.mean() - scores.std()

    def predict(self, X):
        self.X = X[self.features]
        # Compute log-likelihood for each sample.
        scores = self.model.score_samples(self.X)
        # Label as anomaly (1) if the score is below the threshold; normal (0) otherwise.
        return [1 if score < self.threshold else 0 for score in scores]

    def get_name(self):
        return "GaussianMixture"


class IsolationForestWrapper:
    def __init__(self, n_estimators=100, contamination=0.05, features=[]):
        self.model = IsolationForest(
            n_estimators=n_estimators, contamination=contamination
        )
        self.features = features

    def fit(self, X):
        self.X = X[self.features]
        self.model.fit(self.X)

    def predict(self, X):
        self.X = X[self.features]
        # Isolation Forest returns 1 for inliers, -1 for outliers.
        # We map outliers (-1) to anomaly (1) and inliers (+1) to normal (0).
        preds = self.model.predict(self.X)
        return [1 if p == -1 else 0 for p in preds]

    def get_name(self):
        return "IsolationForest"


class MaxRuleBasedWrapper:
    critical_values = {}

    def __init__(self, n=2, percentage=0.10, features=[]):
        self.n = n
        self.percentage = percentage
        self.features = features

    def fit(self, X):
        X = X[self.features]
        features = X.columns

        for feature in features:
            min_val = X[feature].min()
            max_val = X[feature].max()
            self.critical_values[feature] = (min_val, max_val)

    def predict(self, X):
        self.X = X[self.features]
        preds = []

        for i in range(len(self.X)):
            row = self.X.iloc[i]
            outlier_count = 0

            for feature, (min_val, max_val) in self.critical_values.items():
                if row[feature] > max_val * (1 + self.percentage):
                    outlier_count += 1

            if outlier_count >= self.n:
                preds.append(1)
            else:
                preds.append(0)

        return preds

    def get_name(self):
        return "MaxRuleBasedWrapper"


class AverageRuleBasedWrapper:
    average_values = {}

    def __init__(self, n=2, percentage=0.10, features=[]):
        self.n = n
        self.percentage = percentage
        self.features = features

    def fit(self, X):
        self.X = X[self.features]

        for feature in self.features:
            mean_val = X[feature].mean()
            self.average_values[feature] = mean_val

    def predict(self, X):
        self.X = X[self.features]
        preds = []

        for i in range(len(self.X)):
            row = self.X.iloc[i]
            outlier_count = 0

            for feature, mean_val in self.average_values.items():
                if row[feature] > mean_val * (1 + self.percentage):
                    outlier_count += 1

            if outlier_count >= self.n:
                preds.append(1)
            else:
                preds.append(0)

        return preds

    def get_name(self):
        return "AverageRuleBasedWrapper"


class ZScoreWrapper:
    z_scores = {}

    def __init__(self, n=2, threshold=3.0, features=[]):
        self.n = n
        self.threshold = threshold
        self.features = features

    def fit(self, X):
        self.X = X[self.features]

        for feature in self.features:
            mean_val = self.X[feature].mean()
            std_val = self.X[feature].std()
            self.z_scores[feature] = (mean_val, std_val)

    def predict(self, X):
        self.X = X[self.features]
        preds = []

        for i in range(len(self.X)):
            row = self.X.iloc[i]
            outlier_count = 0

            for feature, (mean_val, std_val) in self.z_scores.items():
                z_score = (row[feature] - mean_val) / std_val
                if z_score > self.threshold:
                    outlier_count += 1

            if outlier_count >= self.n:
                preds.append(1)
            else:
                preds.append(0)

        return preds

    def get_name(self):
        return "ZScoreWrapper"

In [ ]:
def evaluate_methods(data, data_labels, methods):
    """
    Evaluate a set of anomaly detection methods with various hyperparameter combinations.

    Parameters:
        data (DataFrame): Preprocessed telemetry data without labels.
        data_labels (Series): The true anomaly labels corresponding to the data (binary values).
        methods (dict): A dictionary where each key is a method name and the value is a dictionary with:
            - 'func': The detector constructor/function (which returns an instance with fit and predict methods)
            - 'hyperparams': (Optional) A dictionary of hyperparameter names mapped to lists of possible values.

    Returns:
        results (list of dict): A list of result dictionaries, each containing:
            - 'method': The method name.
            - 'params': The hyperparameter configuration.
            - 'accuracy', 'recall', 'precision', 'f1': Performance metrics.
            - 'confusion_matrix': The confusion matrix.
        Only the top 10 configurations (by F1 score) for each method are returned.
    """
    results = []

    # Assume these global variables are defined
    global LEARNING_PERIOD_ROWS_COUNT, RETRAINING_PERIOD_ROWS_COUNT, TOTAL_ROWS_COUNT

    for method_name, method_config in methods.items():
        method_func = method_config["func"]
        hyperparams = method_config.get("hyperparams", {})
        features = method_config.get("features", [])

        # Generate all hyperparameter combinations (if hyperparams are provided)
        if hyperparams:
            param_names, param_values = zip(*hyperparams.items())
            hyperparam_combinations = [
                dict(zip(param_names, values)) for values in product(*param_values)
            ]
        else:
            hyperparam_combinations = [{}]

        method_results = []

        for params in hyperparam_combinations:
            # Instantiate the detector using the method constructor and current hyperparameters.
            detector_instance = method_func(features=features, **params)

            # Start training on the learning period
            i = LEARNING_PERIOD_ROWS_COUNT
            detector_instance.fit(data.iloc[:i])
            predicted_labels = []

            training_data = data.iloc[:i].copy()

            # Iterate over remaining rows (simulate real-time prediction)
            while i < TOTAL_ROWS_COUNT:
                # Retrain periodically (after each retraining period)
                if (
                    i - LEARNING_PERIOD_ROWS_COUNT + 1
                ) % RETRAINING_PERIOD_ROWS_COUNT == 0:
                    detector_instance.fit(training_data)

                # Predict anomaly for the current row (assumed to return a list/array)
                anomaly_flags = detector_instance.predict(data.iloc[i : i + 1])
                predicted_labels.append(anomaly_flags[0])
                is_anomaly = anomaly_flags[0] == 1

                if not is_anomaly:
                    training_data = training_data.iloc[1:]
                    training_data = pd.concat(
                        [training_data, data.iloc[[i]]], ignore_index=True
                    )

                i += 1

            # Evaluate performance on the test portion (rows after the learning period)
            y_true = data_labels[LEARNING_PERIOD_ROWS_COUNT:].tolist()
            y_pred = predicted_labels

            f1 = f1_score(y_true, y_pred)
            accuracy = accuracy_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)
            conf_matrix = confusion_matrix(y_true, y_pred)

            result = {
                "method": method_name,
                "params": params,
                "accuracy": accuracy,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "confusion_matrix": conf_matrix,
            }
            method_results.append(result)

        # Sort results for the current method by F1 score (descending) and keep top 3.
        method_results.sort(key=lambda r: r["f1"], reverse=True)
        top_results = method_results[:3]
        results.extend(top_results)

    return results


methods = {
    'IsolationForest': {
        'func': IsolationForestWrapper,
        'hyperparams': {
            'n_estimators': [50, 100, 150],
            'contamination': [0.01, 0.05, 0.1, 0.2],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
        ],
    },
    'OneClassSVM': {
        'func': OneClassSVMWrapper,
        'hyperparams': {
            'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
            'nu': [0.001, 0.01, 0.05, 0.1, 0.15],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
            'gpu_usage',
            'gpu_temperature',
            'gpu_fan_speed',
            'disk_reads',
            'disk_writes',
            'network_bytes_sent',
            'network_bytes_received',
            'network_packets_sent',
            'network_packets_received',
        ],
    },
    'LocalOutlierFactor': {
        'func': LOFWrapper,
        'hyperparams': {
            'n_neighbors': [10, 15, 20, 25, 30],
            'novelty': [True],
            'contamination': [0.01, 0.05, 0.1, 0.2],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
            'gpu_usage',
            'gpu_temperature',
            'gpu_fan_speed',
            'disk_reads',
            'disk_writes',
            'network_bytes_sent',
            'network_bytes_received',
            'network_packets_sent',
            'network_packets_received',
        ],
    },
    'GaussianMixture': {
        'func': GaussianMixtureWrapper,
        'hyperparams': {
            'n_components': [1, 2, 3],
            'covariance_type': ['full', 'diag', 'spherical', 'tied'],
            'threshold': [-3.0, -0.5, -2.0, -2.5, -3.5, -4]  # None means automatic thresholding based on training scores.
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
            'gpu_usage',
            'gpu_temperature',
            'gpu_fan_speed',
            'disk_reads',
            'disk_writes',
            'network_bytes_sent',
            'network_bytes_received',
            'network_packets_sent',
            'network_packets_received',
        ]
    },
    'MaxRuleBased': {
        'func': MaxRuleBasedWrapper,
        'hyperparams': {
            'n': [2, 3, 4],
            'percentage': [0.05, 0.10, 0.15, 0.20],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
            'gpu_usage',
            'gpu_temperature',
            'gpu_fan_speed',
        ],
    },
    'AverageRuleBased': {
        'func': AverageRuleBasedWrapper,
        'hyperparams': {
            'n': [2, 3, 4],
            'percentage': [0.05, 0.10, 0.15, 0.20],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
        ],
    },
    'ZScore': {
        'func': ZScoreWrapper,
        'hyperparams': {
            'n': [1, 2, 3, 4],
            'threshold': [1.5, 2.5, 3.0, 3.5],
        },
        'features': [
            'cpu_usage',
            'cpu_temperature',
            'cpu_speed',
            'cpu_fan_speed',
            'gpu_usage',
            'gpu_temperature',
            'gpu_fan_speed',
            'disk_reads',
            'disk_writes',
            'network_bytes_sent',
            'network_bytes_received',
            'network_packets_sent',
            'network_packets_received',
        ]
    },
    'EllipticEnvelopeWrapper': {
        'func': EllipticEnvelopeWrapper,
        'hyperparams': {
            'contamination': [0.01, 0.05, 0.1, 0.2],
            'support_fraction': [0.5, 0.6, 0.8, 0.9],
        },
        'features': [
            'cpu_usage',
            # 'cpu_temperature',
            'cpu_speed',
            # 'cpu_fan_speed',
            # 'gpu_usage',
            # 'gpu_temperature',
            # 'gpu_fan_speed',
            'disk_reads',
            'disk_writes',
            'network_bytes_sent',
            'network_bytes_received',
            'network_packets_sent',
            'network_packets_received',
        ]
    },
}

results = evaluate_methods(data, data_labels, methods)
# Print top results for each method.
for res in results:
    print("Method:", res['method'])
    print("Hyperparameters:", res['params'])
    print("Accuracy: {:.3f}, Recall: {:.3f}, Precision: {:.3f}, F1: {:.3f}".format(
        res['accuracy'], res['recall'], res['precision'], res['f1']))
    print("Confusion Matrix:\n", res['confusion_matrix'])
    print("---------------------------------------------------")

In [ ]:
def evaluate_final_detector(data, data_labels, final_detectors, threshold_range):
    """
    Evaluate the final anomaly detection decision by aggregating predictions from multiple detectors.

    Parameters:
        data (DataFrame): Preprocessed telemetry data (without labels).
        data_labels (Series): Ground truth binary anomaly labels (0 for normal, 1 for anomaly).
        final_detectors (list): A list of pre-configured detector instances (each with fit() and predict() methods).
        threshold_range (list of int): A list of thresholds (number of detectors required to flag an anomaly) to try.

    Returns:
        results (list of dict): A list of dictionaries, each containing:
            - 'threshold': the aggregation threshold used,
            - 'accuracy', 'recall', 'precision', 'f1': the computed performance metrics,
            - 'confusion_matrix': the confusion matrix.
    """
    # Global variables defined elsewhere:
    # LEARNING_PERIOD_ROWS_COUNT, RETRAINING_PERIOD_ROWS_COUNT, TOTAL_ROWS_COUNT
    # For this function, we assume that the detectors are already initialised with final hyperparameters.

    # Create a copy of the initial training data for sliding-window retraining.
    training_data = data.iloc[:LEARNING_PERIOD_ROWS_COUNT].copy()

    # This list will hold, for every test data point, a list of predictions from all detectors.
    aggregated_predictions = []

    # Fit all detectors on the initial training data.
    for detector in final_detectors:
        detector.fit(training_data)

    # Simulate real-time processing for test data (data points after the learning period).
    for i in range(LEARNING_PERIOD_ROWS_COUNT, TOTAL_ROWS_COUNT):
        # Retrain detectors periodically using the sliding window training data.
        if (i - LEARNING_PERIOD_ROWS_COUNT + 1) % RETRAINING_PERIOD_ROWS_COUNT == 0:
            for detector in final_detectors:
                detector.fit(training_data)

        # For the current data point, collect predictions from each detector.
        current_preds = []
        for detector in final_detectors:
            # Assume each detector.predict returns a list/array; extract the first element.
            pred = detector.predict(data.iloc[i : i + 1])
            current_preds.append(pred[0])
        aggregated_predictions.append(current_preds)

        # Update training_data with current data point if all detectors agree it is normal.
        if all(pred == 0 for pred in current_preds):
            training_data = pd.concat(
                [training_data, data.iloc[[i]]], ignore_index=True
            )
        # Else, if an anomaly is detected, the current point is not added to the training data.

    # Extract ground truth for test period.
    y_true = data_labels.iloc[LEARNING_PERIOD_ROWS_COUNT:].tolist()

    # Evaluate final decision for each threshold.
    results = []
    for thresh in threshold_range:
        # For each test data point, classify as anomaly if number of detectors predicting anomaly >= threshold.
        y_pred_final = [
            1 if sum(preds) >= thresh else 0 for preds in aggregated_predictions
        ]

        # Calculate performance metrics.
        f1 = f1_score(y_true, y_pred_final)
        accuracy = accuracy_score(y_true, y_pred_final)
        recall = recall_score(y_true, y_pred_final)
        precision = precision_score(y_true, y_pred_final)
        conf_matrix = confusion_matrix(y_true, y_pred_final)

        results.append(
            {
                "threshold": thresh,
                "accuracy": accuracy,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "confusion_matrix": conf_matrix,
            }
        )

    return results


final_detectors = []

detector_1 = GaussianMixtureWrapper(
    n_components=3,
    covariance_type="full",
    threshold=-2.0,
    features=[
        "cpu_usage",
        "cpu_speed",
        "disk_reads",
        "disk_writes",
        "network_bytes_sent",
        "network_bytes_received",
        "network_packets_sent",
        "network_packets_received",
    ],
)
detector_2 = AverageRuleBasedWrapper(
    n=4,
    percentage=0.20,
    features=["cpu_usage", "cpu_temperature", "cpu_speed", "cpu_fan_speed"],
)
detector_3 = OneClassSVMWrapper(
    kernel="rbf",
    nu=0.001,
    features=[
        "cpu_usage",
        "cpu_temperature",
        "cpu_speed",
        "cpu_fan_speed",
        "gpu_usage",
        "gpu_temperature",
        "gpu_fan_speed",
    ],
)
detector_4 = ZScoreWrapper(
    n=3,
    threshold=1.5,
    features=[
        "cpu_usage",
        "cpu_speed",
        "disk_reads",
        "disk_writes",
        "network_bytes_sent",
        "network_bytes_received",
        "network_packets_sent",
        "network_packets_received",
    ],
)
detector_5 = MaxRuleBasedWrapper(
    n=2,
    percentage=0.05,
    features=[
        "cpu_usage",
        "cpu_temperature",
        "cpu_speed",
        "cpu_fan_speed",
        "gpu_usage",
        "gpu_temperature",
        "gpu_fan_speed",
    ],
)

final_detectors = [detector_1, detector_2, detector_3, detector_4, detector_5]

threshold_range = list(range(1, len(final_detectors) + 1))

final_results = evaluate_final_detector(
    data, data_labels, final_detectors, threshold_range
)

for res in final_results:
    print(f"Threshold: {res['threshold']}")
    print(
        "Accuracy: {:.3f}, Recall: {:.3f}, Precision: {:.3f}, F1: {:.3f}".format(
            res["accuracy"], res["recall"], res["precision"], res["f1"]
        )
    )
    print("Confusion Matrix:")
    print(res["confusion_matrix"])
    print("---------------------------------------------------")


Threshold: 1
Accuracy: 0.918, Recall: 0.950, Precision: 0.563, F1: 0.707
Confusion Matrix:
[[633  59]
 [  4  76]]
---------------------------------------------------
Threshold: 2
Accuracy: 0.978, Recall: 0.912, Precision: 0.880, F1: 0.896
Confusion Matrix:
[[682  10]
 [  7  73]]
---------------------------------------------------
Threshold: 3
Accuracy: 0.969, Recall: 0.713, Precision: 0.983, F1: 0.826
Confusion Matrix:
[[691   1]
 [ 23  57]]
---------------------------------------------------
Threshold: 4
Accuracy: 0.902, Recall: 0.050, Precision: 1.000, F1: 0.095
Confusion Matrix:
[[692   0]
 [ 76   4]]
---------------------------------------------------
Threshold: 5
Accuracy: 0.898, Recall: 0.013, Precision: 1.000, F1: 0.025
Confusion Matrix:
[[692   0]
 [ 79   1]]
---------------------------------------------------
